In [3]:
counter =1
print(counter,'000')

1 000


In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
from PIL import Image
import skimage as ski

# Define the dataset paths
dataset_path = '/home/salah-eddine/Desktop/Deepfake D/Dataset'
original_train_dir = os.path.join(dataset_path, 'Train')
original_val_dir = os.path.join(dataset_path, 'Validation')
original_test_dir = os.path.join(dataset_path, 'Test')

# Load ESRGAN model from TensorFlow Hub
esrgan_path = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(esrgan_path)

def preprocessing(img):
    # Ensure image has valid dimensions
    if img.shape[0] < 4 or img.shape[1] < 4:
        raise ValueError("Image dimensions must be at least 4x4 pixels.")
    
    imageSize = (tf.convert_to_tensor(img.shape[:-1]) // 4) * 4
    cropped_image = tf.image.crop_to_bounding_box(
        img, 0, 0, imageSize[0], imageSize[1])
    preprocessed_image = tf.cast(cropped_image, tf.float32)
    return tf.expand_dims(preprocessed_image, 0)

def srmodel(img):
    preprocessed_image = preprocessing(img)  
    new_image = model(preprocessed_image)  
    return tf.squeeze(new_image)

# Preprocessing: Detect faces and save to new directories
preprocessed_base = '/home/salah-eddine/Desktop/Deepfake D/Output'
preprocessed_train_dir = os.path.join(preprocessed_base, 'Train')
preprocessed_val_dir = os.path.join(preprocessed_base, 'Validation')
preprocessed_test_dir = os.path.join(preprocessed_base, 'Test')

# Create directories for preprocessed images
os.makedirs(preprocessed_train_dir, exist_ok=True)
os.makedirs(preprocessed_val_dir, exist_ok=True)
os.makedirs(preprocessed_test_dir, exist_ok=True)

for class_name in ['Real', 'Fake']:
    os.makedirs(os.path.join(preprocessed_train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_val_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(preprocessed_test_dir, class_name), exist_ok=True)

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_save_faces(original_dir, preprocessed_dir):
    for class_name in ['Real', 'Fake']:
        class_path = os.path.join(original_dir, class_name)
        preprocessed_class_path = os.path.join(preprocessed_dir, class_name)
        
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            img = cv2.imread(image_path)
            if img is not None:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
                
                if len(faces) > 0:
                    # Select largest face
                    x, y, w, h = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)[0]
                    face = img[y:y+h, x:x+w]
                    # Ensure face is in RGB format
                    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

                    output_path = os.path.join(preprocessed_class_path, image_name)

                    if w >= 256 and h >= 256:
                        # Save the original-sized face image
                        Image.fromarray(face).save(output_path)
                        print(f"Saved original-sized face: {output_path}")
                    else:
                        # Resize the image using the SR model
                        face = tf.convert_to_tensor(face, dtype=tf.float32)
                        face = srmodel(face)
                        # Convert back to uint8 and save
                        face = tf.clip_by_value(face, 0, 255)
                        face = tf.cast(face, tf.uint8).numpy()
                        # apply Gaussian blur, creating a new image
                        blurred = ski.filters.gaussian(image, sigma=(sigma, sigma), truncate=3.5, channel_axis=-1)
                        Image.fromarray(face).save(output_path)
                        print(f"Saved upscaled face: {output_path}")
            else:
                print(f"Warning: Could not read image {image_path}")

# Process images and save faces
print("Preprocessing training data...")
detect_and_save_faces(original_train_dir, preprocessed_train_dir)
print("Preprocessing validation data...")
detect_and_save_faces(original_val_dir, preprocessed_val_dir)
print("Preprocessing test data...")
detect_and_save_faces(original_test_dir, preprocessed_test_dir)


2025-02-16 18:07:42.113968: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-16 18:07:42.116667: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-16 18:07:42.124936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739725662.139619    1816 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739725662.143745    1816 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 18:07:42.157717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Preprocessing training data...
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_40439.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_63826.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_28126.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_52099.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_68633.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_23856.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_60332.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_17692.jpg
Saved upscaled and sharpened face: /home/salah-eddine/Desktop/Deepfake D/Output/Train/Real/real_41779.jpg
Saved upscaled 